# Definitions

In [1]:
import pandas as pd

def fasta_to_dataframe(fasta_file):
    headers = []
    sequences = []
    with open(fasta_file, 'r') as f:
        current_sequence = []
        current_header = None
        for line in f:
            line = line.strip()
            if line.startswith('>'):  # Header line
                if current_header is not None:
                    # Save the previous sequence
                    headers.append(current_header)
                    sequences.append(''.join(current_sequence))
                current_header = line[1:]  # Remove '>'
                current_sequence = []
            else:
                current_sequence.append(line)
        # Add the last sequence
        if current_header is not None:
            headers.append(current_header)
            sequences.append(''.join(current_sequence))
    # Create a DataFrame
    return pd.DataFrame({'Header': headers, 'Sequence': sequences})

## Read and Format FASTA file

In [2]:
fasta_file = "../../data/DB.COX1.trimmed.fna"
cutadapt_result = fasta_to_dataframe(fasta_file)

cutadapt_result[['BOLD Metadata', 'Taxonomy']] = cutadapt_result['Header'].str.split(';', n=1, expand=True)
cutadapt_result[['BOLD ID', 'Read ID', 'Country']] = cutadapt_result['BOLD Metadata'].str.split('|', n=2, expand=True)
cutadapt_result['BOLD ID'] = cutadapt_result['BOLD ID'].str.removeprefix('BOLD:')

tax_columns = ['Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
cutadapt_result[tax_columns] = cutadapt_result['Taxonomy'].str.extract(
    r'p:([^,]+),c:([^,]+),o:([^,]+),f:([^,]+),g:([^,]+),s:([^;]+)'
)

cutadapt_result = cutadapt_result.drop(columns=['Header', 'BOLD Metadata', 'Taxonomy'])
print(cutadapt_result.head())

                                            Sequence  BOLD ID       Read ID  \
0  TTTATCCTCTAACATTGCTCATGGAGGGTCTTCTGTAGATCTAGCT...  AAA7085  BLPDA1021-18   
1  TTTATCCTCTAACATTGCTCATGGAGGGTCTTCTGTAGATCTAGCT...  AAA7085  MHMYN6354-14   
2  TTTATCCTCTAACATTGCTCATGGAGGGTCTTCTGTAGATCTAGCT...  AAA7085   MHASB734-07   
3  TTTATCCTCTAACATTGCTCATGGAGGGTCTTCTGTAGATCTAGCT...  AAA7085   MHATB310-06   
4  TTTATCCTCTAACATTGCTCATGGAGGGTCTTCTGTAGATCTAGCT...  AAA7085  BLPAA6663-17   

      Country      Phylum    Class        Order       Family    Genus  \
0  Costa_Rica  Arthropoda  Insecta  Lepidoptera  Saturniidae  Lonomia   
1  Costa_Rica  Arthropoda  Insecta  Lepidoptera  Saturniidae  Lonomia   
2  Costa_Rica  Arthropoda  Insecta  Lepidoptera  Saturniidae  Lonomia   
3  Costa_Rica  Arthropoda  Insecta  Lepidoptera  Saturniidae  Lonomia   
4  Costa_Rica  Arthropoda  Insecta  Lepidoptera  Saturniidae  Lonomia   

                 Species  
0  Lonomia_santarosensis  
1  Lonomia_santarosensis  
2  Lo

## Reading my Result

In [3]:
my_result = pd.read_csv("../../data/current/primer-finder-fixed3.csv", sep=";")

my_result['f_index'] = my_result['f_index'].astype(int)
my_result['b_index'] = my_result['b_index'].astype(int)
my_result['f_match'] = my_result['f_match'].astype(str)

my_result['region'] = my_result.apply(lambda x: x['read'][(x['f_index'] + len(x['f_match'])):x['b_index']], axis=1)
my_result['region_length'] = my_result['region'].str.len()
print(my_result.head())

         BOLD ID       Read ID     Country            Phylum      Class  \
0  >BOLD:AAA7085  BLPDA1021-18  Costa_Rica  tax=p:Arthropoda  c:Insecta   
1  >BOLD:AAA7085  MHMYN6354-14  Costa_Rica  tax=p:Arthropoda  c:Insecta   
2  >BOLD:AAA7085   MHASB734-07  Costa_Rica  tax=p:Arthropoda  c:Insecta   
3  >BOLD:AAA7085   MHATB310-06  Costa_Rica  tax=p:Arthropoda  c:Insecta   
4  >BOLD:AAA7085  BLPAA6663-17  Costa_Rica  tax=p:Arthropoda  c:Insecta   

           Order         Family      Genus                  Species  f_score  \
0  o:Lepidoptera  f:Saturniidae  g:Lonomia  s:Lonomia_santarosensis       49   
1  o:Lepidoptera  f:Saturniidae  g:Lonomia  s:Lonomia_santarosensis       49   
2  o:Lepidoptera  f:Saturniidae  g:Lonomia  s:Lonomia_santarosensis       49   
3  o:Lepidoptera  f:Saturniidae  g:Lonomia  s:Lonomia_santarosensis       49   
4  o:Lepidoptera  f:Saturniidae  g:Lonomia  s:Lonomia_santarosensis       49   

                      f_match  f_index  b_score                  b_m

## Compare Mine and Cutadapt
* result of my perfect matches with cutadapt result
* filter all which dont exist in cutadapt
* what else?

#### Are there any areas which differ for perfect matches?

In [4]:
perfect_matches = my_result[~((my_result['f_score'] != 52) | (my_result['b_score'] != 46))]
filtered_df = cutadapt_result[cutadapt_result['Read ID'].isin(perfect_matches['Read ID'])]

In [5]:
print(f'Perfect matches: {perfect_matches.shape[0]}')
print(f'Perfect matches found in cutadapt result: {filtered_df.shape[0]}')

dif = perfect_matches.shape[0] - filtered_df.shape[0]
print(f'Diff: {dif}, or {dif / perfect_matches.shape[0] * 100}% were not found in the cutadapt result.\n')

Perfect matches: 898561
Perfect matches found in cutadapt result: 898561
Diff: 0, or 0.0% were not found in the cutadapt result.



In [6]:
merged_df = filtered_df[['Read ID', 'Sequence']].merge(
    perfect_matches[['Read ID', 'region']],
    on='Read ID',
    how='inner'
).rename(columns={
    'Sequence': 'ca_sequence',
    'region': 'my_sequence'
})
different_sequences = merged_df[~(merged_df['ca_sequence'] == merged_df['my_sequence'])]
print(different_sequences.shape[0])
print(different_sequences.head())

28
              Read ID                                        ca_sequence  \
153755  GBMNE23810-21  TTTATCTGGAGGTATTGCCCATGGGGGTGCTTCCGTAGATTTAGCT...   
684885    NEPTA859-13  CTTATCAGCAAATATTGCTCATAGTGGTAGATCAGTTGATTTAGCA...   
684898    NEPTA862-13  CTTATCAGCAAATATTGCTCATAGTGGTAGATCAGTTGATTTAGCA...   
696620    GCOL7710-16  TTTATCTTCTAATATTACTCATAGAGGAGCTTCTGTTGATTTAGCT...   
696621    GENHP169-11  TTTATCTTCTAATATTACTCATAGAGGAGCTTCTGTTGATTTAGCT...   

                                              my_sequence  
153755  TTTATCTGGAGGTATTGCCCATGGGGGTGCTTCCGTAGATTTAGCT...  
684885  CTTATCAGCAAATATTGCTCATAGTGGTAGATCAGTTGATTTAGCA...  
684898  CTTATCAGCAAATATTGCTCATAGTGGTAGATCAGTTGATTTAGCA...  
696620  TTTATCTTCTAATATTACTCATAGAGGAGCTTCTGTTGATTTAGCT...  
696621  TTTATCTTCTAATATTACTCATAGAGGAGCTTCTGTTGATTTAGCT...  


In [7]:
different_sequences['ca_length'] = different_sequences['ca_sequence'].str.len()
different_sequences['my_length'] = different_sequences['my_sequence'].str.len()
# 28 / 898561 * 100

C:\Users\Julian\AppData\Local\Temp\ipykernel_8804\3086744800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  different_sequences['ca_length'] = different_sequences['ca_sequence'].str.len()
C:\Users\Julian\AppData\Local\Temp\ipykernel_8804\3086744800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  different_sequences['my_length'] = different_sequences['my_sequence'].str.len()


#### why do those (28) entires fail with cutadapt?

In [8]:
perfect_failed = my_result[my_result['Read ID'].isin(different_sequences['Read ID'])]
perfect_failed = different_sequences.merge(
    perfect_failed,
    on='Read ID',
    how='inner'
).drop(columns=['region', 'region_length'])

In [9]:
perfect_failed.to_csv('../../data/current/perfect_failed.csv', index=False)

#### whats up with my bad results, when one result was perfect??

In [10]:
should_be_good = my_result[~((my_result['f_score'] != 52) & (my_result['b_score'] != 46))]
filtered_df2 = cutadapt_result[cutadapt_result['Read ID'].isin(should_be_good['Read ID'])]
merged_df2 = should_be_good.merge(
    filtered_df2,
    on='Read ID',
    how='inner'
).rename(columns={
    'Sequence': 'ca_sequence',
    'region': 'my_sequence'
})

merged_df2['ca_length'] = merged_df2['ca_sequence'].str.len()
merged_df2['my_length'] = merged_df2['my_sequence'].str.len()

In [11]:
merged_with_diff = merged_df2[merged_df2['ca_sequence'] != merged_df2['my_sequence']]
#merged_df2_buckets = merged_with_diff.groupby(['b_score']).size().reset_index(name='Count')
#merged_df2_buckets
merged_with_diff = merged_with_diff[(merged_with_diff['my_length'] <= 185) & (merged_with_diff['my_length'] >= 140)].sort_values('my_length', ascending=False)
merged_with_diff['remainder'] = merged_with_diff.apply(lambda x: len(x['read'][x['f_index']+231::]), axis=1)
merged_with_diff.groupby(['remainder']).size().reset_index()


,remainder,0
0,0,471
1,1,23
2,2,1
3,3,1817
4,4,213
5,5,215
6,6,5173
7,7,164
8,8,531
9,9,2216


In [12]:
merged_df

,Read ID,ca_sequence,my_sequence
0,SPMNP492-22,ACTTTCATCTAATATTGCTCATAACGGATCTTCTGTTGATTTAGCT...,ACTTTCATCTAATATTGCTCATAACGGATCTTCTGTTGATTTAGCT...
1,SPTMA398-07,TTTATCCTCAAATATTGCCCATAGTGGAAGATCTGTTGATTTAGCT...,TTTATCCTCAAATATTGCCCATAGTGGAAGATCTGTTGATTTAGCT...
2,INSPH096-19,TTTATCCTCAAATATTGCCCATAGTGGAAGATCTGTTGATTTAGCT...,TTTATCCTCAAATATTGCCCATAGTGGAAGATCTGTTGATTTAGCT...
3,SPTMA557-09,TTTATCCTCAAATATTGCCCATAGTGGAAGATCTGTTGATTTAGCT...,TTTATCCTCAAATATTGCCCATAGTGGAAGATCTGTTGATTTAGCT...
4,INSPH172-20,TTTATCCTCAAATATTGCCCATAGTGGAAGATCTGTTGATTTAGCT...,TTTATCCTCAAATATTGCCCATAGTGGAAGATCTGTTGATTTAGCT...
...,...,...,...
898556,GBMNA32630-19,TCTTTCATCCGGTATTGCCCATGCTGGTGGATCTGTAGATTTAGCT...,TCTTTCATCCGGTATTGCCCATGCTGGTGGATCTGTAGATTTAGCT...
898557,LEPMY2719-22,TTTATCATCTAACATTGCTCATAGAGGAAGATCAGTAGATTTAGCT...,TTTATCATCTAACATTGCTCATAGAGGAAGATCAGTAGATTTAGCT...
898558,LIMBC627-11,ACTTTCATCTAACATTGCCCATAGAGGAAGATCTGTTGACTTAGCA...,ACTTTCATCTAACATTGCCCATAGAGGAAGATCTGTTGACTTAGCA...
898559,SARBF2798-16,TCTCTCAGCTAACATTGCCCATAGCGGCTCTTCAGTTGACTTAGCC...,TCTCTCAGCTAACATTGCCCATAGCGGCTCTTCAGTTGACTTAGCC...


#### does cutadapt always cut at least 3 base pairs? - yes:

In [13]:
merged_with_remainder = my_result.merge(
    cutadapt_result,
    on='Read ID',
    how='inner'
)
merged_with_remainder['remainder'] = merged_with_remainder[merged_with_remainder['Sequence'] != ''].apply(
    lambda x: len(x['read'].split(x['Sequence'])[-1]), axis=1
)

merged_with_remainder.groupby(['remainder']).size().reset_index(name='Count').sort_values(by=['remainder'], ascending=[True])

,remainder,Count
0,3.0,3332
1,4.0,462
2,5.0,409
3,6.0,9200
4,7.0,786
...,...,...
905,1029.0,1
906,1056.0,16
907,1066.0,88
908,1071.0,1


#### what is the best score, if the b-primer is outside the read?
there are regions with len 190 lol
also, with a remainder of 12 or less, there might be a b-primer match in front of the f-primer match, resulting in NO region/sequence.

In [14]:
outside = my_result
outside['remainder'] = outside.apply(lambda x: len(x['read'][x['f_index']+231::]), axis=1)
outside = outside[outside['remainder'] <= 12]

outside_no_region = outside[outside['region'] == ''].copy()
outside_no_region['region'] = outside_no_region.apply(lambda x: x['read'][(x['f_index'] + len(x['f_match'])):x['b_index']], axis=1)
outside_no_region

,BOLD ID,Read ID,Country,Phylum,Class,Order,Family,Genus,Species,f_score,f_match,f_index,b_score,b_match,b_index,read,region,region_length,remainder
16593,>BOLD:AAC9112,BBCNP186-13,Canada,tax=p:Arthropoda,c:Arachnida,o:Araneae,f:Linyphiidae,g:Bathyphantes,s:Bathyphantes_pallidus,32,GGAGCTGGTTGGACTGTAT,319,27,CCTATTTTAATTGGGGGTTTT,166,GACGTTATATTTAATTTTTGGTGCCTGGGCAGCTATGGTAGGGACT...,,0,0
19620,>BOLD:AAB6956,VVGAP469-20,Ukraine,tax=p:Arthropoda,c:Insecta,o:Coleoptera,f:Curculionidae,g:Larinus,s:Larinus_minutus,31,GTACTTTGACTGTATA-TGTATATCCA,552,27,CCACTCCTGGCTGGGTCGTTCAATTA,26,TGCGGTCCTCGGATTCTGTTCCCGGACCACTCCTGGCTGAGGGTCG...,,0,0
22005,>BOLD:AAM1247,MAHEM349-10,Pakistan,tax=p:Arthropoda,c:Insecta,o:Hemiptera,f:Aleyrodidae,g:Bemisia,s:Bemisia_tabaci,29,GGTATCTCGTCTATTCAGATTATCCT,589,28,CTTATTTTACCAGGCTGGTATT,19,TCATCCAGAAGTTTATGTTCTTATTTTACCAGGCTTTGGTATTGTT...,,0,0
22368,>BOLD:AAM1247,MAHEM382-10,Pakistan,tax=p:Arthropoda,c:Insecta,o:Hemiptera,f:Aleyrodidae,g:Bemisia,s:Bemisia_tabaci,29,GGTATCTCGTCTATTCAGATTATCCT,589,28,CTTATTTTACCAGGCTGGTATT,19,TCATCCAGAAGTTTATGTTCTTATTTTACCAGGCTTTGGTATTGTT...,,0,9
22481,>BOLD:AAM1247,MAHEM370-10,Pakistan,tax=p:Arthropoda,c:Insecta,o:Hemiptera,f:Aleyrodidae,g:Bemisia,s:Bemisia_tabaci,29,GGTATCTCGTCTATTCAGATTATCCT,589,28,CTTATTTTACCAGGCTGGTATT,19,TCATCCAGAAGTTTATGTTCTTATTTTACCAGGCTTTGGTATTGTT...,,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378502,>BOLD:ADC8118,GBMTG4732-16,NaN,tax=p:Arthropoda,c:Arachnida,o:Ixodida,f:Argasidae,g:Argas,s:Argas_miniatus,29,GG---TGGTTCGGAACAAGTTATATCGT,782,30,CCGTGATTTATCAGGGAG-AATCA,588,ATTTCTTTAATAAGATATATATTATTGCTTGTTATAGTGTTGATTA...,,0,0
2379746,>BOLD:ADR7996,DISA506-19,South_Atlantic_Ocean,tax=p:Arthropoda,c:Malacostraca,o:Isopoda,f:Sphaeromatidae,g:Exosphaeroma,s:Exosphaeroma_truncatitelson,30,GAATTGAAATGACCTGTACAAACCCTC,434,29,CCCTGATATTCAAGCCGCAG-TATCCAC,38,AAAACATCGTCGGTGTCTACTTTCGCCGATAAGATCTGCCCAGTGA...,,0,0
2380340,>BOLD:ACL6603,ZSMDB022-14,Chile,tax=p:Arthropoda,c:Insecta,o:Coleoptera,f:Dytiscidae,g:Rhantus,s:Rhantus_signatus,30,GGTACGCAGGAAATAACTGATTTAATGCCT,603,33,CATGTATTAGCTCTAG-AATTAC,559,CACCCGGCAGGACGTCTCGCGGGCGTGCCAGTTTAACACCGCGGGC...,,0,0
2380872,>BOLD:AAJ5917,GBLN0644-06,NaN,tax=p:Arthropoda,c:Insecta,o:Lepidoptera,f:Nymphalidae,g:Aemona,s:Aemona_lena,31,GGGTCAAGGAT---CTGGTATATCTACCACC,215,28,CCGAGTTTTCGATCTAGGTGACTTTTA,31,CTGCACTGTGAAGACGTGTTGGATGCGTCTGCCGAGTTTTCGATCT...,,0,0


In [1]:
cutadapt_result[cutadapt_result['Read ID'] == "MAHEM382-10"]

NameError: name 'cutadapt_result' is not defined